## Imports

In [ ]:

%load_ext autoreload
%autoreload 2

import os
import sys
import pickle
import numpy as np
import pprint as pp
import pysindy as ps
from pathlib import Path
from scipy import fft
from scipy.integrate import solve_ivp
from scipy.signal import savgol_coeffs, savgol_filter
from sklearn.preprocessing import MinMaxScaler

# Ignore matplotlib deprecation warnings
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

# Seed the random number generators for reproducibility
np.random.seed(100)

# Update path to include mypkg
mypkg_path = str(Path(os.path.abspath('')).parent.parent.parent.absolute())
sys.path.insert(0, mypkg_path)

from src import helpers, plot_data, global_config, datasets
config = global_config.config
image_dir_og = config.top_dir



## Load data (and smooth it)

In [ ]:
# Hyperparameters

# Initial condition parameters
n_avg = 10 # number of curves for moving average
u_true_cutoff = 800 # final index for MNIST propogated wave
u_true_start = 60

# IVP parameters
dx = None
x = None
t = None
dt = None


In [ ]:

# Load the data
dataset = "STL10"
config.top_dir = str(Path(image_dir_og).parent / "fft_images" / dataset)
file = os.path.join(config.top_dir, "pkl", f"{dataset.lower()}_og_integral.pkl")
with open(file, 'rb') as file:
    d = pickle.load(file)

# Calculate moving average
u_total = helpers.moving_average(d['ints'],n=n_avg,axis=0)
u_true = u_total[u_true_start:u_true_cutoff, :]
x = np.asarray(d['r_x'])
t_total, t_true = (np.arange(0,u_total.shape[0]) + 0., np.arange(0,u_true.shape[0]) + 0.)
dt = 1.


In [ ]:

# Plots
plot_data.plot_surface(
        z=u_total,x=x,y=t_total,
        xaxis_title="x", yaxis_title="time", zaxis_title="u(t,x)",
        title='PDE Input (Full Data)',
        hovertemplate='t: %{y:0.2f}<br>x: %{x:0.2f}<br> u: %{z:0.2f}<extra></extra>',
        colorscale='agsunset'
    )

plot_data.plot_surface(
        z=u_true, x=x, y=t_total,
        xaxis_title="x", yaxis_title="time", zaxis_title="u(t,x)",
        title='PDE Input (Cropped Data)',
        hovertemplate='t: %{y:0.2f}<br>x: %{x:0.2f}<br> u: %{z:0.2f}<extra></extra>',
        colorscale='agsunset'
    )

# Plot slices of u
xs = [x,x,x]
ys = [u_true[0,:], u_true[30,:], u_true[-1,:]]
labels = [f"u(x,t=0)", f"u(x,t=30)", f"u(x,t={u_true.shape[0]-1})"]
title = "Time slices of u(x,t)"

plot_data.plot_line(x=xs, y=ys, label=labels, title=title)



## Smoothing (Savitzky Golay)

In [ ]:

u_interp = list()
t=60
u_savgol = savgol_filter(u_true[t,:], 4, 3, mode='nearest')

xs = [x,x]
ys = [u_true[t,:], u_savgol]
labels = [f"u(x,t={t})", f"u_savgol(x,t={t})"]
title = "Savgol filter example"

plot_data.plot_line(x=xs, y=ys, label=labels, title=title)


In [ ]:

u_interp = list()
x_interp = x

for t_idx in range(u_true.shape[0]):
    u_savgol = savgol_filter(u_true[t_idx,:], 4, 3, mode='nearest')
    u_interp.append(u_savgol)
u_interp = np.stack(u_interp)

plot_data.plot_surface(
        z=u_interp, x=x_interp, y=t_true,
        xaxis_title="x", yaxis_title="time", zaxis_title="u(t,x)",
        title='PDE Input (Smoothed with SG)',
        hovertemplate='t: %{y:0.2f}<br>x: %{x:0.2f}<br> u: %{z:0.2f}<extra></extra>',
        colorscale='agsunset'
    )


## Rescale

In [ ]:

x_scaled, x_min, x_max = helpers.min_max_fit(x_interp)
t_scaled, t_min, t_max = helpers.min_max_fit(t_true)
u_scaled, u_min, u_max = helpers.min_max_fit(u_interp)

plot_data.plot_surface(
        z=u_scaled, x=x_scaled, y=t_scaled,
        xaxis_title="x", yaxis_title="time", zaxis_title="u(t,x)",
        title='PDE Input (Cropped, SG, and Scaled Data)',
        hovertemplate='t: %{y:0.2f}<br>x: %{x:0.2f}<br> u: %{z:0.2f}<extra></extra>',
        colorscale='agsunset'
    )

x_inv = helpers.min_max_fit_inv(x_scaled, x_min, x_max, 0., 1.,)
t_inv = helpers.min_max_fit_inv(t_scaled, t_min, t_max, 0., 1.,)
u_inv = helpers.min_max_fit_inv(u_scaled, u_min, u_max, 0., 1.,)

print("||u_inv - u_true||_2", np.linalg.norm(u_inv-u_interp))


## Feature Library

In [ ]:

u = u_scaled; x = x_scaled; t = t_scaled;
dx = x[1]-x[0]; dt = t[1]-t[0]

dummy_u = np.random.randn(x.shape[0], t.shape[0], 1)

# Define PDE library that is quadratic in u, and
# third-order in spatial derivatives of u.
# library_functions = [lambda x: x, lambda x: x * x]
# library_function_names = [lambda x: x, lambda x: x + x]
pde_lib = ps.PDELibrary(function_library=ps.PolynomialLibrary(degree=2,include_bias=False),
                        derivative_order=4, spatial_grid=x,
                        include_bias=True, is_uniform=True)

dummy_pde_lib = pde_lib
dummy_pde_lib.fit([dummy_u])
feature_names = [helpers.modify_pde_sindy_out(feature) for feature in dummy_pde_lib.get_feature_names()]
print("Library:")
print(feature_names)


## Grid Search

In [ ]:

md = helpers.pysindy_grid_search(pde_lib, u_scaled, x_scaled, t_scaled, 0.8, 50, 180, 1e-5)
print()
pp.pprint(md)
